# Homework 02: Non-systematic state space traversal - 2+A - Genetic Algorithm for N queens problem

We will try  to place $N$ queens on $N$ by $N$ chessboard, such that no queen attacks other, that is none of them share vertical, horizontal or diagonal line.

## Implementation
To implement Genetic Algorithm we need to define:
* Problem encoding - how to represent queens' positions
* Loss/Fitness function - measure that we will try to optimize
* Selection - to propagate best individuals further into generations
* Crossover - to mimic evolutions genom crossing
* Mutation  - to increase diversity and escape local minima

All of the above except *Selection* will be encapsulated in **Individual** class.

Our chromosome will consist of $N$ numbers in range $[0, N-1] \cap \mathbb{Z}$ that will represent a queen at position $(i, c_i)$, where $i$ is row on which the queen is located and $c_i$ is $i$-th entry in chromosome. This encoding does not allow for queens to share rows, nevertheless it is enough to represent correct solution due to our restraints.

We will use number of attacking pairs as loss function. If it is zero, then no queen attacks the other and we have solved it. Therefore our goal is to minimize number of attacking pairs.

For crossover we will choose one-point strategy where we will swap prefixes of random length of two chromosomes.

Mutations will be parametrized by number of trials and probability of mutation. On each trial with ceratin probability random entry of chromosome will be changed to other number in gene-range.

In [1]:
from copy import deepcopy
from random import randint, seed, random

class Individual:
    
    QUEEN_CHAR = 'Q'
    TILE_CHAR  = '·'
    
    def __init__(self, N : int):
        self.N = N
        self.chromosome = [randint(0, N-1) for _ in range(N)]
        self._attacks = [0 for _ in range(N)]
        self.loss = 0
        self.compute_loss()

    def __repr__(self) -> str:
        ret = ''
        for queen in self.queens:
            ret += Individual.TILE_CHAR * queen + Individual.QUEEN_CHAR + Individual.TILE_CHAR * (self.N - queen - 1) + '\n'
        return ret

    def __xor__(lhs, rhs):
        """
            Crossover operator
        """
        assert lhs.N == rhs.N
        new_l = deepcopy(lhs)
        new_r = deepcopy(rhs)
        bound = randint(0, lhs.N - 1)
        new_l.chromosome[:bound], new_r.chromosome[:bound] = new_r.chromosome[:bound], new_l.chromosome[:bound]
        new_l.compute_loss()
        new_r.compute_loss()
        return new_l, new_r

    def compute_loss(self):
        for idx, queen in enumerate(self.chromosome):
            self._attacks[idx] = 0
            for jdx, pueen in enumerate(self.chromosome[(idx+1):], start=idx+1):
                dx = queen - pueen
                if dx == 0:
                    self._attacks[idx] += 1
                    continue
                dy = idx - jdx
                if dx == dy or dx == -dy:
                    self._attacks[idx] += 1
        self.loss = sum(self._attacks)
        return self.loss

    def mutate(self, probability, n_mutations):
        for _ in range(n_mutations):
            if random() < probability:
                self.chromosome[randint(0, self.N - 1)] = randint(0, self.N - 1)
        return self

    def display_attacks(self) -> str:
        ret = f'Loss: {self.loss}\n'
        for queen, attacks in zip(self.chromosome, self._attacks):
            ret += Individual.TILE_CHAR * queen + str(attacks) + Individual.TILE_CHAR * (self.N - queen - 1) + '\n'
        return ret

We will implelemt tournament selection i.e. we will sample fixed amount of individuals from whole population and choose the one with the best score, in our case the least loss.

In [2]:
from random import sample

def select(population:list[Individual], tournament_size):
    return min(sample(population, tournament_size), key=lambda x:x.loss)

def check_population(population:list[Individual]):
    for ind in population:
        if ind.loss == 0:
            return True
    return False

def NQueens(N : int, n_individuals:int=64, tournament_size:int=8, max_gens:int=200, mutate_probability=0.5, n_mutations=1):
    assert tournament_size < n_individuals
    population = [Individual(N) for _ in range(n_individuals)]
    generation = 0

    while not check_population(population) and generation < max_gens:
        next_gen = []
        for _ in range(n_individuals//2):
            a = select(population, tournament_size)
            b = select(population, tournament_size)
            x, y = a ^ b
            x.mutate(mutate_probability, n_mutations)
            y.mutate(mutate_probability, n_mutations)
            next_gen.append(x)
            next_gen.append(y)
        population = next_gen
        generation += 1
    population.sort(key=lambda x:x.loss)
    return generation, population

## Running algorithm

We will seed before every call, so it will reproducible.

### N = 8

In [3]:
%%time
seed(0)
gen, pop = NQueens(
    8,
    mutate_probability=0.7
)
print('Generation: ', gen)
print(pop[0].display_attacks())

Generation:  80
Loss: 0
······0·
··0·····
·······0
·0······
····0···
0·······
·······0
···0····

CPU times: user 310 ms, sys: 3.7 ms, total: 314 ms
Wall time: 308 ms


It solved regular 8x8 board in less that a second.

### N = 10

In [4]:
%%time
seed(0)
gen, pop = NQueens(
    10, mutate_probability=0.7
)
print('Generation: ', gen)
print(pop[0].display_attacks())

Generation:  113
Loss: 0
······0···
····0·····
··0·······
········0·
·····0····
·········0
·0········
·0········
0·········
·······0··

CPU times: user 353 ms, sys: 847 μs, total: 354 ms
Wall time: 354 ms


Result is very similiar to one for $N = 8$

### N = 20

Starting here, algorithm not only benefits from, but requires fine-tunning or it does not solve N-Queens for sensible amount of generations.

These settings were picked specifically for ```seed(0)```.

In [5]:
%%time
seed(0)
gen, pop = NQueens(
    20, 
    mutate_probability=0.8,
    n_mutations=2,
    n_individuals=128,
    tournament_size=16
)
print('Generation: ', gen)
print(pop[0].display_attacks())

Generation:  57
Loss: 0
·············0······
·····0··············
·········0··········
···0················
······0·············
···················0
·················0··
···············0····
·0··················
·······0············
····0···············
··········0·········
0···················
················0···
···0················
···········0········
··0·················
·················0··
··················0·
··············0·····

CPU times: user 692 ms, sys: 812 μs, total: 693 ms
Wall time: 691 ms


Algorithm still managed to solve it under two seconds.

### N = 50

This one required a lot more tunning, than any before.

In [6]:
%%time
seed(0)
gen, pop = NQueens(
    50,
    mutate_probability = 0.8,
    n_mutations=2,
    n_individuals=256,
    tournament_size=16,
    max_gens=400
)
print('Generation: ', gen)
print(pop[0].display_attacks())

Generation:  377
Loss: 0
······················0···························
·····································0············
································0·················
·····························0····················
0·················································
···············0··································
··········0·······································
··················0·······························
·······················0··························
··································0···············
·····0············································
······························0···················
···································0··············
···········0······································
······································0···········
···········································0······
·······0··········································
·········0········································
·················································0
······

This time took more half a minute and almost 400 generations to find solution. As size of board grows it gets harder to escape local minima, so naturally we try to increase mutations and it takes longer to stabilize and find best solution.

### N = 100

We were not able to find settings, under which loss equals zero, but we were able to drive it extremly low.

In [7]:
%%time
seed(0)
gen, pop = NQueens(
    100,
    mutate_probability = 0.7,
    n_mutations=3,
    n_individuals=512,
    tournament_size=64,
    max_gens=600
)
print('Generation: ', gen)
print(pop[0].display_attacks())

Generation:  600
Loss: 2
·····························0······································································
·····0······························································································
··················································0·················································
······················································0·············································
·························································································0··········
································································································0···
·················································0··················································
············································································0·······················
····························································0·······································
··········································0·······················

## Conclusion

We were able to reach some level of success with genetic algorithm for N-Queens resolving up to $N=50$. Though, some improvements can be made. Algorithm can be modified, we can choose different crossover and mutation tactics or try to use chane-selection to bring more diversity. Computational difficulty is another obstacle, so implementing Genetic Algorithm in faster language like Rust or Cython, would definetly allow for greater number generations and thus better results.